# **Transfer Learning**
---
## 쉽지만 중요한 코너!
## **"잘 만들어진"** 모델들을 가져다가 **"고쳐"** 사용해보자.

* [모델들은 이 링크를 참고해보자](https://keras.io/api/applications/)
* [트랜스퍼 러닝?](https://miro.medium.com/max/2800/1*D5S6ylZwUZAxj0lMyipZ2g.png)

![transfer](https://raw.githubusercontent.com/DrKAI/image/main/Transfer_Learning.png)


## Keras Upgrade

In [ ]:
!pip install keras-nightly

## Library Loading

In [ ]:
import keras

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from keras.preprocessing import image
from keras.utils import to_categorical
from keras.layers import GlobalAveragePooling2D, Dense

from sklearn.model_selection import train_test_split

import random
import numpy as np
import matplotlib.pyplot as plt
import glob

## **Collecting Image Data**

* 최소 조건 : 클래스 3개, 한 클래스당 10장 이상. **다다익선!**
    * ImageNet data에는 확실히 없을만한 것들로. (이를테면 만화 캐릭터간 비교)
    * **좋은 결과**를 위해서라면 클래스가 확실히 차이나는 이미지로.
    * **도전(역경)**을 위해서라면 클래스가 달라도 비슷비슷해 보이는 이미지로.
---
- **순서**
    1. 구글링하여 이미지를 수집합니다.
    2. **본인의** 구글 드라이브에 my_data 폴더를 생성합니다.
    3. my_data 폴더 안에 transfer 폴더를 생성합니다.
    4. 1번 단계에서 수집한 이미지를 transfer 폴더 안에 업로드하되, **하나의 클래스당 하나의 폴더를 갖도록 정리합니다.**
    5. 30초 정도 기다립니다.
    5. 아래 코드들을 실행합니다.

## Connect Colaboratory with my Google Drive
- Colaboratory와 본인의 구글 드라이브를 연결하는 과정
- 아래 코드를 실행하여 폴더가 올바르게 생성 되었는지 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cd /content/drive/MyDrive/my_data/; ls

In [ ]:
!cd /content/drive/MyDrive/my_data/transfer; ls

In [ ]:
files = glob.glob('/content/drive/MyDrive/my_data/transfer/*/*')
files

In [ ]:
files[-1].split('/')[-2]

In [ ]:
name_cnt = {}

for x in files :
    name_cnt[x.split('/')[-2]] = name_cnt.get(x.split('/')[-2], 0) + 1

name_cnt

In [ ]:
i = 0
names = {}

for key in name_cnt :
    names[key] = i     # names_cnt의 key값에 새로운 값 부여
    i += 1             # 클래스 수만큼 i값 증가

names

In [ ]:
images = []
labels = []

for path in files:
    img = image.load_img(path, target_size=(299,299) )
    img = image.img_to_array(img)

    images.append(img)
    labels.append(names[path.split('/')[-2]])

    plt.imshow(image.load_img(path))
    plt.show()

images_arr = np.array(images)
labels_arr = np.array(labels)

print(images_arr.shape)
print(labels_arr.shape)

In [ ]:
print(labels_arr)
label_v = len(np.unique(labels_arr))
label_v

In [ ]:
### 라벨링
y = to_categorical(labels, label_v)

In [ ]:
print(y[:3])
y.shape

## Dataset split

- training set, validation set, test set 생성
- 각 이미지 그룹별로 균등한 분할을 위하여. 아래 코드가 조금 복잡합니다.

In [ ]:
name_cnt.values()

In [ ]:
temp = []
init_v = 0

for v in name_cnt.values() :
    temp.append( (images[init_v:init_v+v], y[init_v:init_v+v]) )
    init_v += v

In [ ]:
for i in range(len(temp)) :
    x_to_array = np.array(temp[i][0])
    y_to_array = np.array(temp[i][1])

    train_x, test_x, train_y, test_y =\
        train_test_split(x_to_array, y_to_array, test_size=0.2, random_state=2024)

    train_x, valid_x, train_y, valid_y =\
        train_test_split(train_x, train_y, test_size=0.2, random_state=2024)

    if i==0 :
        first_tr_x, first_va_x, first_te_x = train_x.copy(), valid_x.copy(), test_x.copy()
        first_tr_y, first_va_y, first_te_y = train_y.copy(), valid_y.copy(), test_y.copy()

    elif i==1 :
        new_tr_x, new_tr_y = np.vstack((first_tr_x, train_x)), np.vstack((first_tr_y, train_y))
        new_va_x, new_va_y = np.vstack((first_va_x, valid_x)), np.vstack((first_va_y, valid_y))
        new_te_x, new_te_y = np.vstack((first_te_x, test_x)), np.vstack((first_te_y, test_y))

    else :
        new_tr_x, new_tr_y = np.vstack((new_tr_x, train_x)), np.vstack((new_tr_y, train_y))
        new_va_x, new_va_y = np.vstack((new_va_x, valid_x)), np.vstack((new_va_y, valid_y))
        new_te_x, new_te_y = np.vstack((new_te_x, test_x)), np.vstack((new_te_y, test_y))

In [ ]:
new_tr_x.shape, new_tr_y.shape, new_va_x.shape, new_va_y.shape, new_te_x.shape, new_te_y.shape

In [ ]:
# 전처리 하지 않은 파일 따로 시각화 해두기
train_xv, valid_xv, test_xv = train_x.copy(), valid_x.copy(), test_x.copy()

## Data Preprocessing
- **잘 만들어진** 모델에서 제공하는 전처리 과정을 사용합니다.

In [ ]:
new_tr_x.max(), new_tr_x.min()

In [ ]:
new_tr_x = preprocess_input(new_tr_x)
new_va_x = preprocess_input(new_va_x)
new_te_x = preprocess_input(new_te_x)

In [ ]:
new_tr_x.max(), new_tr_x.min()

## Load **Pretrained** Model

In [ ]:
keras.backend.clear_session()

base_model = InceptionV3(weights='imagenet',       # ImageNet 데이터를 기반으로 미리 학습된 가중치 불러오기
                         include_top=False,        # InceptionV3 모델의 아웃풋 레이어는 제외하고 불러오기
                         input_shape= (299,299,3)) # 입력 데이터의 형태

new_output = GlobalAveragePooling2D()(base_model.output)
new_output = Dense(3, # class 3개   클래스 개수만큼 진행한다.
                  activation = 'softmax')(new_output)

model = keras.models.Model(base_model.inputs, new_output)

model.summary()

In [ ]:
from keras.utils import plot_model

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
print(f'모델의 레이어 수 : {len(model.layers)}')

## **Fine-tuning**
- 모델의 가중치를 그대로 사용할 레이어와 추가 학습할 레이어를 결정합니다.

In [ ]:
model.layers

In [ ]:
for idx, layer in enumerate(model.layers) :
    if idx < 213 :
        layer.trainable = False
    else :
        layer.trainable = True

In [ ]:
# 처음부터 학습시키는 것도 아니고,
# 마지막 100개의 레이어만 튜닝 할 것이므로 learning rate를 조금 크게 잡아본다.

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
             optimizer=keras.optimizers.Adam(learning_rate=0.001) )

## Image Data Augmentation & Callbacks

In [ ]:
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                 patience=4,
                                 verbose=1,
                                 factor=0.5,
                                 min_lr=0.000001)

es = EarlyStopping(monitor='val_loss',
                   min_delta=0, # 개선되고 있다고 판단하기 위한 최소 변화량
                   patience=4,  # 개선 없는 epoch 얼마나 기다려 줄거야
                   verbose=1,
                   restore_best_weights=True)

## .fit( )

In [ ]:
# 데이터를 넣어서 학습시키자!
hist = model.fit(train_x, train_y,
                 validation_data=(valid_x, valid_y),
                 epochs=1000, verbose=1,
                 callbacks=[es, lr_reduction]
                 )

## Result

In [ ]:
model.evaluate(test_x, test_y) ## [loss, accuracy]

In [ ]:
y_pred = model.predict(test_x)
y_pred

In [ ]:
to_names = { v:k for k,v in names.items() }

In [ ]:
for i in range(len(test_x)) :
    print('------------------------------------------------------')
    print(f'실제 정답 : {to_names[test_y[i].argmax()]} vs 모델의 예측 : {to_names[y_pred[i].argmax()]} ')
    prob = ''

    for j in to_names :
        string = f'{to_names[j]} : {y_pred[i][j]*100:.2f}%  '
        prob = prob + string
    print(prob)
    plt.imshow(test_xv[i].reshape([299,299,3])/255)
    plt.show()